# Post-Processing Evaluation Notebook

**Purpose:** Re-evaluate existing batch processing results using configurable evaluation metrics.

**Workflow:**
1. Load batch results CSV files (contain extracted field values)
2. Load ground truth data
3. Re-evaluate using selected metric(s):
   - Position-aware F1 (strict position matching)
   - Correlation-aware F1 (validates alignment across related fields)
   - Position-agnostic F1 (ignores order)
4. Generate new CSV files compatible with model_comparison.ipynb

**Benefits:**
- Test different evaluation approaches without re-running model inference
- Compare metric sensitivities
- Identify which metric best reflects true extraction quality

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path

# Add project root to path (notebook is in root directory)
project_root = Path.cwd()
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Tuple
from rich.console import Console
from rich.table import Table
from rich.progress import track

# Import evaluation utilities
from common.evaluation_metrics import (
    load_ground_truth,
    calculate_field_accuracy_with_method,
    calculate_field_accuracy_f1,
)
from common.config import get_document_type_fields

console = Console()
console.print("[green]✅ Imports successful[/green]")

## 2. Configuration

**Configure the specific batch result CSV files you want to evaluate.**

Specify exact file paths instead of using auto-discovery.

In [ ]:
# ============================================================================
# INPUT CONFIGURATION - EDIT THESE PATHS
# ============================================================================

# Batch result CSV files to evaluate
# Format: {'model_label': 'path/to/batch_results.csv'}
# Set path to None or remove entry to skip that model
BATCH_RESULT_FILES = {
    'llama': 'output/csv/llama_batch_results_20251024_224027.csv',
    'internvl3_quantized': 'output/csv/internvl3_batch_results_20251024_224904.csv',
    # 'internvl3_non_quantized': 'output/csv/internvl3_non_quantized_batch_results_20251024_224549.csv',
}

# Ground truth CSV path
GROUND_TRUTH_CSV = 'evaluation_data/ground_truth.csv'

# Evaluation method selection
# Options: 'order_aware_f1', 'f1' (position-agnostic), 'correlation', 'all'
EVALUATION_METHOD = 'all'  # Change to specific method or keep 'all' for comparison

# Output directory
OUTPUT_DIR = project_root / 'output' / 'csv' / 'post_evaluation'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Timestamp for output files
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')

# ============================================================================

console.print(f"[cyan]Ground truth CSV: {GROUND_TRUTH_CSV}[/cyan]")
console.print(f"[cyan]Output directory: {OUTPUT_DIR}[/cyan]")
console.print(f"[cyan]Evaluation method: {EVALUATION_METHOD}[/cyan]")
console.print(f"[cyan]Timestamp: {TIMESTAMP}[/cyan]")

## 3. Load Batch Result Files

In [ ]:
def load_batch_results(file_path: str | Path) -> pd.DataFrame:
    """Load batch results CSV and parse list fields."""
    file_path = Path(file_path)
    
    if not file_path.is_absolute():
        file_path = project_root / file_path
    
    if not file_path.exists():
        raise FileNotFoundError(f"Batch result file not found: {file_path}")
    
    df = pd.read_csv(file_path, dtype=str)
    
    # Replace NaN with "NOT_FOUND" for consistency
    df = df.fillna("NOT_FOUND")
    
    console.print(f"[cyan]Loaded {len(df)} rows from {file_path.name}[/cyan]")
    
    return df


# Load all configured batch results
batch_dataframes = {}

for model_name, file_path in BATCH_RESULT_FILES.items():
    if file_path is None:
        console.print(f"[yellow]Skipping {model_name} (no file path configured)[/yellow]")
        continue
    
    try:
        batch_dataframes[model_name] = load_batch_results(file_path)
        console.print(f"[green]✅ Loaded {model_name} from {Path(file_path).name}[/green]")
    except FileNotFoundError as e:
        console.print(f"[red]❌ {e}[/red]")
    except Exception as e:
        console.print(f"[red]❌ Error loading {model_name}: {e}[/red]")

if not batch_dataframes:
    console.print("[red]❌ No batch result files loaded![/red]")
else:
    console.print(f"\n[green]✅ Loaded {len(batch_dataframes)} batch result dataframes[/green]")

## 4. Load Ground Truth Data

In [ ]:
# Load ground truth
ground_truth_path = Path(GROUND_TRUTH_CSV)
if not ground_truth_path.is_absolute():
    ground_truth_path = project_root / ground_truth_path

if not ground_truth_path.exists():
    console.print(f"[red]❌ Ground truth file not found: {ground_truth_path}[/red]")
    ground_truth_data = None
else:
    ground_truth_data = load_ground_truth(
        str(ground_truth_path),
        show_sample=True,
        verbose=True
    )
    console.print(f"[green]✅ Loaded ground truth for {len(ground_truth_data)} images[/green]")

## 5. Re-Evaluation Engine

In [ ]:
def re_evaluate_extraction(
    extracted_fields: Dict[str, str],
    ground_truth: Dict[str, str],
    document_type: str,
    evaluation_method: str = 'order_aware_f1'
) -> Dict:
    """
    Re-evaluate extracted fields against ground truth using specified method.
    
    Args:
        extracted_fields: Dictionary of {field_name: value}
        ground_truth: Dictionary of {field_name: value}
        document_type: Document type (for field selection)
        evaluation_method: 'order_aware_f1', 'f1', or 'correlation'
    
    Returns:
        Dictionary with overall metrics and field-level scores
    """
    # Get document-specific fields
    doc_type_normalized = document_type.lower().replace(' ', '_')
    fields_to_evaluate = get_document_type_fields(doc_type_normalized)
    
    field_scores = {}
    total_f1 = 0.0
    total_precision = 0.0
    total_recall = 0.0
    fields_evaluated = 0
    fields_matched = 0
    
    for field_name in fields_to_evaluate:
        extracted_value = extracted_fields.get(field_name, "NOT_FOUND")
        ground_truth_value = ground_truth.get(field_name, "NOT_FOUND")
        
        # Skip if both are NOT_FOUND
        if extracted_value == "NOT_FOUND" and ground_truth_value == "NOT_FOUND":
            continue
        
        fields_evaluated += 1
        
        # Calculate field accuracy with chosen method
        try:
            metrics = calculate_field_accuracy_with_method(
                extracted_value,
                ground_truth_value,
                field_name,
                method=evaluation_method,
                debug=False
            )
        except Exception as e:
            console.print(f"[yellow]⚠ Error evaluating {field_name}: {e}[/yellow]")
            metrics = {'f1_score': 0.0, 'precision': 0.0, 'recall': 0.0}
        
        field_scores[field_name] = metrics
        total_f1 += metrics.get('f1_score', 0.0)
        total_precision += metrics.get('precision', 0.0)
        total_recall += metrics.get('recall', 0.0)
        
        # Count as matched if F1 > 0.9
        if metrics.get('f1_score', 0.0) > 0.9:
            fields_matched += 1
    
    # Calculate overall metrics
    overall_accuracy = (total_f1 / fields_evaluated * 100) if fields_evaluated > 0 else 0.0
    overall_precision = (total_precision / fields_evaluated) if fields_evaluated > 0 else 0.0
    overall_recall = (total_recall / fields_evaluated) if fields_evaluated > 0 else 0.0
    
    return {
        'overall_accuracy': overall_accuracy,
        'overall_precision': overall_precision,
        'overall_recall': overall_recall,
        'fields_evaluated': fields_evaluated,
        'fields_matched': fields_matched,
        'field_scores': field_scores
    }


console.print("[green]✅ Re-evaluation function defined[/green]")

In [ ]:
def process_batch_with_method(
    df: pd.DataFrame,
    ground_truth_data: Dict,
    method: str,
    model_name: str
) -> pd.DataFrame:
    """
    Re-evaluate entire batch using specified method.
    
    Args:
        df: Batch results dataframe
        ground_truth_data: Ground truth dictionary
        method: Evaluation method name
        model_name: Model identifier
    
    Returns:
        DataFrame with updated evaluation metrics
    """
    # Create a copy to avoid modifying original
    result_df = df.copy()
    
    # Field columns to extract
    field_columns = [
        'DOCUMENT_TYPE', 'BUSINESS_ABN', 'SUPPLIER_NAME', 'BUSINESS_ADDRESS',
        'PAYER_NAME', 'PAYER_ADDRESS', 'INVOICE_DATE', 'LINE_ITEM_DESCRIPTIONS',
        'LINE_ITEM_QUANTITIES', 'LINE_ITEM_PRICES', 'LINE_ITEM_TOTAL_PRICES',
        'IS_GST_INCLUDED', 'GST_AMOUNT', 'TOTAL_AMOUNT', 'STATEMENT_DATE_RANGE',
        'TRANSACTION_DATES', 'TRANSACTION_AMOUNTS_PAID', 'TRANSACTION_AMOUNTS_RECEIVED',
        'ACCOUNT_BALANCE'
    ]
    
    console.print(f"\n[bold cyan]Re-evaluating {model_name} with {method}[/bold cyan]")
    
    # Process each row
    for idx in track(range(len(result_df)), description=f"Processing {model_name}"):
        row = result_df.iloc[idx]
        image_name = row['image_name']
        document_type = row.get('document_type', 'INVOICE')
        
        # Get ground truth for this image
        gt = ground_truth_data.get(image_name)
        if not gt:
            console.print(f"[yellow]⚠ No ground truth for {image_name}[/yellow]")
            continue
        
        # Extract fields from row
        extracted_fields = {}
        for field in field_columns:
            if field in row.index:
                extracted_fields[field] = row[field]
        
        # Re-evaluate
        evaluation = re_evaluate_extraction(
            extracted_fields,
            gt,
            document_type,
            evaluation_method=method
        )
        
        # Update metrics in dataframe
        result_df.at[idx, 'overall_accuracy'] = f"{evaluation['overall_accuracy']:.2f}"
        result_df.at[idx, 'fields_matched'] = str(evaluation['fields_matched'])
        result_df.at[idx, 'total_fields'] = str(evaluation['fields_evaluated'])
    
    return result_df


console.print("[green]✅ Batch processing function defined[/green]")

## 6. Execute Re-Evaluation

In [ ]:
# Define evaluation methods to run
if EVALUATION_METHOD == 'all':
    methods_to_run = ['order_aware_f1', 'f1', 'correlation']
    method_labels = {
        'order_aware_f1': 'Position-Aware F1',
        'f1': 'Position-Agnostic F1',
        'correlation': 'Correlation-Aware F1'
    }
else:
    methods_to_run = [EVALUATION_METHOD]
    method_labels = {EVALUATION_METHOD: EVALUATION_METHOD.replace('_', ' ').title()}

console.print(f"[bold green]Running evaluation with methods: {methods_to_run}[/bold green]")

In [ ]:
# Store results for each model and method
evaluated_results = {}

if ground_truth_data is None:
    console.print("[red]❌ Cannot evaluate without ground truth data[/red]")
else:
    for model_name, df in batch_dataframes.items():
        evaluated_results[model_name] = {}
        
        for method in methods_to_run:
            console.rule(f"[bold]{model_name} - {method_labels.get(method, method)}[/bold]")
            
            # Re-evaluate with this method
            result_df = process_batch_with_method(
                df,
                ground_truth_data,
                method,
                model_name
            )
            
            evaluated_results[model_name][method] = result_df
            
            # Show summary
            avg_accuracy = result_df['overall_accuracy'].astype(float).mean()
            console.print(f"[green]✅ {method_labels.get(method, method)}: Average accuracy = {avg_accuracy:.2f}%[/green]")

console.print("\n[bold green]🎉 Re-evaluation complete![/bold green]")

## 7. Generate Output CSV Files

In [ ]:
# Save re-evaluated results
output_files = {}

for model_name, method_results in evaluated_results.items():
    output_files[model_name] = {}
    
    for method, result_df in method_results.items():
        # Generate output filename
        method_suffix = method.replace('_', '-')
        output_filename = f"{model_name}_{method_suffix}_evaluated_{TIMESTAMP}.csv"
        output_path = OUTPUT_DIR / output_filename
        
        # Save CSV
        result_df.to_csv(output_path, index=False)
        output_files[model_name][method] = output_path
        
        console.print(f"[green]💾 Saved: {output_filename}[/green]")

console.print(f"\n[bold green]✅ All output files saved to: {OUTPUT_DIR}[/bold green]")

## 8. Comparison Analysis

In [ ]:
def compare_evaluation_methods(
    model_name: str,
    method_results: Dict[str, pd.DataFrame]
) -> pd.DataFrame:
    """
    Compare accuracy across different evaluation methods for a model.
    """
    comparison_data = []
    
    for method, df in method_results.items():
        accuracies = df['overall_accuracy'].astype(float)
        
        comparison_data.append({
            'Method': method_labels.get(method, method),
            'Mean Accuracy': f"{accuracies.mean():.2f}%",
            'Median Accuracy': f"{accuracies.median():.2f}%",
            'Std Dev': f"{accuracies.std():.2f}",
            'Min': f"{accuracies.min():.2f}%",
            'Max': f"{accuracies.max():.2f}%"
        })
    
    return pd.DataFrame(comparison_data)


# Generate comparison tables
if EVALUATION_METHOD == 'all':
    console.print("\n[bold cyan]Evaluation Method Comparison[/bold cyan]")
    
    for model_name, method_results in evaluated_results.items():
        console.rule(f"[bold]{model_name}[/bold]")
        
        comparison_df = compare_evaluation_methods(model_name, method_results)
        
        # Display as Rich table
        table = Table(title=f"{model_name} - Method Comparison")
        for col in comparison_df.columns:
            table.add_column(col, style="cyan")
        
        for _, row in comparison_df.iterrows():
            table.add_row(*[str(v) for v in row.values])
        
        console.print(table)
        
        # Save comparison to CSV
        comparison_file = OUTPUT_DIR / f"{model_name}_method_comparison_{TIMESTAMP}.csv"
        comparison_df.to_csv(comparison_file, index=False)
        console.print(f"[green]💾 Saved comparison: {comparison_file.name}[/green]\n")

## 9. Document Type Breakdown

In [ ]:
def analyze_by_document_type(
    model_name: str,
    method: str,
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Analyze accuracy breakdown by document type.
    """
    if 'document_type' not in df.columns:
        return None
    
    doc_type_stats = []
    
    for doc_type in df['document_type'].unique():
        type_df = df[df['document_type'] == doc_type]
        accuracies = type_df['overall_accuracy'].astype(float)
        
        doc_type_stats.append({
            'Document Type': doc_type,
            'Count': len(type_df),
            'Mean Accuracy': f"{accuracies.mean():.2f}%",
            'Median Accuracy': f"{accuracies.median():.2f}%",
            'Std Dev': f"{accuracies.std():.2f}"
        })
    
    return pd.DataFrame(doc_type_stats)


# Generate document type analysis
console.print("\n[bold cyan]Document Type Breakdown[/bold cyan]")

for model_name, method_results in evaluated_results.items():
    for method, result_df in method_results.items():
        console.rule(f"[bold]{model_name} - {method_labels.get(method, method)}[/bold]")
        
        doctype_df = analyze_by_document_type(model_name, method, result_df)
        
        if doctype_df is not None:
            # Display as Rich table
            table = Table(title=f"{model_name} - {method_labels.get(method, method)}")
            for col in doctype_df.columns:
                table.add_column(col, style="cyan")
            
            for _, row in doctype_df.iterrows():
                table.add_row(*[str(v) for v in row.values])
            
            console.print(table)
            console.print()

## 10. Summary and Next Steps

In [ ]:
console.rule("[bold green]Summary[/bold green]")

console.print(f"\n[cyan]Processed models: {', '.join(evaluated_results.keys())}[/cyan]")
console.print(f"[cyan]Evaluation methods: {', '.join([method_labels.get(m, m) for m in methods_to_run])}[/cyan]")
console.print(f"[cyan]Output directory: {OUTPUT_DIR}[/cyan]")

console.print("\n[bold green]Next Steps:[/bold green]")
console.print("1. Review the generated CSV files in the output directory")
console.print("2. Use these files with model_comparison.ipynb for visualization")
console.print("3. Compare how different evaluation metrics affect model rankings")
console.print("4. Identify which metric best aligns with true extraction quality")

console.print("\n[bold green]🎉 Post-processing evaluation complete![/bold green]")